<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE3-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enhanced MLP Agent - Integrated with VAE Pipeline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

class EnhancedMLPAgent:
    """
    Enhanced MLP Agent that integrates seamlessly with VAE pipeline
    Features: Smart data loading, incremental training, comprehensive evaluation
    """

    def __init__(self, output_dir='/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'):
        self.output_dir = output_dir
        self.model = None
        self.transformer = StandardScaler()
        self.is_fitted = False

        # Create MLP-specific directory
        self.mlp_dir = f"{output_dir}MLP_Models/"
        self.checkpoint_dir = f"{self.mlp_dir}checkpoints/"
        os.makedirs(self.mlp_dir, exist_ok=True)
        os.makedirs(self.checkpoint_dir, exist_ok=True)

    def load_vae_outputs(self, data_file=None, windows_file=None, auto_detect=True):
        """
        Smart loading of VAE outputs with auto-detection
        """
        print("="*60)
        print("LOADING VAE OUTPUTS")
        print("="*60)

        if auto_detect:
            # Try to find the most recent VAE outputs - Updated file patterns
            possible_files = [
                # Latest optimized files
                (f'{self.output_dir}generated-data-OPTIMIZED.npy', f'{self.output_dir}generated-data-true-window-OPTIMIZED.npy'),
                # Current generation files
                (f'{self.output_dir}generated_large_subsquence_data.npy', f'{self.output_dir}generated-data-true-window-OPTIMIZED.npy'),
                # Legacy files from your earlier runs
                (f'{self.output_dir}generated-data2.npy', f'{self.output_dir}generated-data-true-window2.npy'),
                (f'{self.output_dir}generated-data.npy', f'{self.output_dir}generated-data-true-window.npy'),
                # Check if the generation is still in progress - use existing synthetic data
                (f'{self.output_dir}generated_large_subsquence_data.npy', None),  # Just synthetic data
            ]

            print("🔍 Searching for VAE output files...")

            # Debug: List all files in directory
            try:
                all_files = [f for f in os.listdir(self.output_dir) if f.endswith('.npy')]
                print(f"📁 Found .npy files in directory:")
                for f in sorted(all_files):
                    file_path = os.path.join(self.output_dir, f)
                    size_mb = os.path.getsize(file_path) / (1024*1024)
                    print(f"   {f} ({size_mb:.1f} MB)")
            except Exception as e:
                print(f"   Could not list directory: {e}")

            # Try to find matching files
            for data_path, windows_path in possible_files:
                if os.path.exists(data_path):
                    if windows_path is None or os.path.exists(windows_path):
                        print(f"✅ Auto-detected VAE outputs:")
                        print(f"   Data: {os.path.basename(data_path)}")
                        if windows_path:
                            print(f"   Windows: {os.path.basename(windows_path)}")
                        else:
                            print("   Windows: Searching for compatible window file...")
                            # Look for any window file that might match
                            window_patterns = [
                                f'{self.output_dir}generated-data-true-window-OPTIMIZED.npy',
                                f'{self.output_dir}generated-data-true-window2.npy',
                                f'{self.output_dir}generated-data-true-window.npy'
                            ]
                            for wp in window_patterns:
                                if os.path.exists(wp):
                                    windows_path = wp
                                    print(f"   Found windows: {os.path.basename(windows_path)}")
                                    break

                            if windows_path is None:
                                print("   ❌ No compatible window file found")
                                continue

                        data_file, windows_file = data_path, windows_path
                        break

            if data_file is None:
                print("❌ No VAE outputs found!")
                print("\n💡 Possible solutions:")
                print("   1. Run the VAE pipeline first:")
                print("      generator = VAEDataGenerator()")
                print("      results = generator.run_smart_pipeline(data_path)")
                print("   2. Or specify file paths manually:")
                print("      mlp_agent.load_vae_outputs(data_file='path/to/data.npy', windows_file='path/to/windows.npy')")
                return None, None

        # Load the data
        try:
            print("Loading synthetic data and VAR windows...")
            x = np.load(data_file)
            y = np.load(windows_file)

            print(f"✅ Successfully loaded:")
            print(f"   Synthetic data: {x.shape}")
            print(f"   VAR windows: {y.shape}")

            # Data quality checks
            print(f"\n📊 Data Quality Check:")
            print(f"   Data range: [{np.min(x):.4f}, {np.max(x):.4f}]")
            print(f"   Windows range: [{np.min(y)}, {np.max(y)}]")
            print(f"   Contains NaN: Data={np.isnan(x).any()}, Windows={np.isnan(y).any()}")

            return x, y

        except Exception as e:
            print(f"❌ Error loading VAE outputs: {e}")
            return None, None

    def analyze_data_distribution(self, y):
        """Analyze and visualize target distribution"""
        print("\n📈 Target Distribution Analysis:")

        # Basic statistics
        print(f"   Mean: {np.mean(y):.2f}")
        print(f"   Std: {np.std(y):.2f}")
        print(f"   Min: {np.min(y)}, Max: {np.max(y)}")
        print(f"   Unique values: {len(np.unique(y))}")

        # Plot distribution
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.hist(y, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        plt.title('VAR Window Distribution')
        plt.xlabel('Optimal VAR Window Size')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 2, 2)
        unique_vals, counts = np.unique(y, return_counts=True)
        plt.bar(unique_vals, counts, alpha=0.7, color='lightcoral')
        plt.title('VAR Window Value Counts')
        plt.xlabel('Window Size')
        plt.ylabel('Count')
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def remove_outliers(self, x, y, contamination=0.1, method='isolation_forest'):
        """Enhanced outlier removal with multiple methods"""
        print(f"\n🔍 Removing outliers using {method}...")
        original_size = len(x)

        if method == 'isolation_forest':
            iso = IsolationForest(contamination=contamination, random_state=42)
            outlier_labels = iso.fit_predict(x)
            mask = outlier_labels != -1
        elif method == 'iqr':
            # IQR method for target variable
            Q1 = np.percentile(y, 25)
            Q3 = np.percentile(y, 75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            mask = (y >= lower_bound) & (y <= upper_bound)
        else:
            print("Unknown method, skipping outlier removal")
            return x, y

        x_clean, y_clean = x[mask], y[mask]
        removed_count = original_size - len(x_clean)

        print(f"   Removed {removed_count} outliers ({removed_count/original_size*100:.1f}%)")
        print(f"   Remaining samples: {len(x_clean)}")

        return x_clean, y_clean

    def prepare_data(self, x, y, test_size=0.1, validation_size=0.1):
        """Enhanced data preparation with train/val/test split"""
        print("\n⚙️ Preparing data...")

        # Transform target
        y_transformed = self.transformer.fit_transform(y.reshape(-1,1)).flatten()
        print(f"   Target transformed: mean={np.mean(y_transformed):.4f}, std={np.std(y_transformed):.4f}")

        # Three-way split: train/val/test
        x_temp, x_test, y_temp, y_test = train_test_split(
            x, y_transformed, test_size=test_size, random_state=42
        )

        val_size_adjusted = validation_size / (1 - test_size)
        x_train, x_val, y_train, y_val = train_test_split(
            x_temp, y_temp, test_size=val_size_adjusted, random_state=42
        )

        print(f"   Data split:")
        print(f"     Train: {x_train.shape[0]} samples ({x_train.shape[0]/len(x)*100:.1f}%)")
        print(f"     Validation: {x_val.shape[0]} samples ({x_val.shape[0]/len(x)*100:.1f}%)")
        print(f"     Test: {x_test.shape[0]} samples ({x_test.shape[0]/len(x)*100:.1f}%)")

        return x_train, x_val, x_test, y_train, y_val, y_test

    def build_mlp(self, input_dim, architecture='default'):
        """Enhanced MLP builder with multiple architectures"""
        print(f"🏗️ Building MLP with {architecture} architecture...")

        model = Sequential()

        if architecture == 'default':
            # Your original architecture
            model.add(Dense(64, activation='relu', input_dim=input_dim))
            model.add(Dense(32, activation='relu'))
            model.add(Dense(16, activation='relu'))
            model.add(Dense(8, activation='relu'))
            model.add(Dense(1))
        elif architecture == 'deep':
            model.add(Dense(128, activation='relu', input_dim=input_dim))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(32, activation='relu'))
            model.add(Dense(16, activation='relu'))
            model.add(Dense(8, activation='relu'))
            model.add(Dense(1))
        elif architecture == 'wide':
            model.add(Dense(256, activation='relu', input_dim=input_dim))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(1))

        optimizer = keras.optimizers.Adam(learning_rate=0.0003, clipnorm=1)
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

        print(f"   Total parameters: {model.count_params():,}")
        return model

    def train_initial_mlp(self, x_train, y_train, x_val, y_val, epochs=1000, batch_size=64, architecture='default'):
        """Enhanced initial training with validation monitoring"""
        print("\n🚀 Training initial MLP...")

        # Build model
        self.model = self.build_mlp(x_train.shape[1], architecture)

        # Callbacks
        checkpoint_path = f"{self.mlp_dir}best_model.weights.h5"
        callbacks = [
            ModelCheckpoint(
                checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            ),
            EarlyStopping(
                patience=50,
                verbose=1,
                min_delta=0.0001,
                monitor='val_loss',
                mode='min',
                restore_best_weights=True
            )
        ]

        # Training
        history = self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        self.is_fitted = True
        print("✅ Initial MLP training complete!")

        # Plot training history
        self.plot_training_history(history)

        return history

    def plot_training_history(self, history):
        """Plot training and validation metrics"""
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 2)
        plt.plot(history.history['mae'], label='Training MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title('Model MAE')
        plt.ylabel('MAE')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 3)
        # Learning rate (if available)
        if 'lr' in history.history:
            plt.plot(history.history['lr'], label='Learning Rate')
            plt.title('Learning Rate')
            plt.ylabel('LR')
            plt.xlabel('Epoch')
            plt.yscale('log')
            plt.legend()
            plt.grid(True, alpha=0.3)
        else:
            plt.text(0.5, 0.5, 'No LR data', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Learning Rate (N/A)')

        plt.tight_layout()
        plt.show()

    def evaluate_model_comprehensive(self, x_test, y_test, dataset_name="Test"):
        """Comprehensive model evaluation with multiple metrics"""
        print(f"\n📊 Comprehensive {dataset_name} Evaluation:")

        # Predictions
        y_pred_raw = self.model.predict(x_test, verbose=0)
        y_pred = self.transformer.inverse_transform(y_pred_raw)
        y_true = self.transformer.inverse_transform(y_test.reshape(-1,1)).flatten()

        # Metrics
        r2 = r2_score(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)

        # Additional metrics
        mape = np.mean(np.abs((y_true - y_pred.flatten()) / y_true)) * 100
        accuracy_within_1 = np.mean(np.abs(y_true - y_pred.flatten()) <= 1) * 100
        accuracy_within_2 = np.mean(np.abs(y_true - y_pred.flatten()) <= 2) * 100

        print(f"   R² Score: {r2:.4f}")
        print(f"   MSE: {mse:.4f}")
        print(f"   MAE: {mae:.4f}")
        print(f"   RMSE: {rmse:.4f}")
        print(f"   MAPE: {mape:.2f}%")
        print(f"   Accuracy within ±1: {accuracy_within_1:.1f}%")
        print(f"   Accuracy within ±2: {accuracy_within_2:.1f}%")

        # Visualization
        self.plot_predictions(y_true, y_pred.flatten(), dataset_name)

        return {
            'r2': r2, 'mse': mse, 'mae': mae, 'rmse': rmse, 'mape': mape,
            'accuracy_1': accuracy_within_1, 'accuracy_2': accuracy_within_2,
            'y_true': y_true, 'y_pred': y_pred.flatten()
        }

    def plot_predictions(self, y_true, y_pred, title="Predictions"):
        """Enhanced prediction visualization"""
        plt.figure(figsize=(15, 10))

        # Scatter plot
        plt.subplot(2, 3, 1)
        plt.scatter(y_true, y_pred, alpha=0.5, s=1)
        plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.title(f'{title} - Scatter Plot')
        plt.grid(True, alpha=0.3)

        # Residuals
        plt.subplot(2, 3, 2)
        residuals = y_true - y_pred
        plt.scatter(y_pred, residuals, alpha=0.5, s=1)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predictions')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)

        # Error distribution
        plt.subplot(2, 3, 3)
        plt.hist(residuals, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        plt.xlabel('Residuals')
        plt.ylabel('Frequency')
        plt.title('Error Distribution')
        plt.grid(True, alpha=0.3)

        # Time series comparison (sample)
        plt.subplot(2, 3, 4)
        sample_size = min(200, len(y_true))
        indices = np.random.choice(len(y_true), sample_size, replace=False)
        indices = np.sort(indices)
        plt.plot(y_true[indices], 'b-', label='True', linewidth=1)
        plt.plot(y_pred[indices], 'r--', label='Predicted', linewidth=1)
        plt.xlabel('Sample Index')
        plt.ylabel('Window Size')
        plt.title(f'{title} - Sample Comparison')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Error by predicted value
        plt.subplot(2, 3, 5)
        abs_errors = np.abs(residuals)
        plt.scatter(y_pred, abs_errors, alpha=0.5, s=1)
        plt.xlabel('Predictions')
        plt.ylabel('Absolute Error')
        plt.title('Absolute Error vs Prediction')
        plt.grid(True, alpha=0.3)

        # Accuracy bands
        plt.subplot(2, 3, 6)
        errors = [0.5, 1, 1.5, 2, 2.5, 3]
        accuracies = [np.mean(np.abs(residuals) <= err) * 100 for err in errors]
        plt.bar(errors, accuracies, alpha=0.7, color='lightgreen')
        plt.xlabel('Error Tolerance')
        plt.ylabel('Accuracy (%)')
        plt.title('Accuracy at Different Tolerances')
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def save_model_with_metadata(self, model_name="enhanced_mlp_model", metadata=None):
        """Save model with comprehensive metadata"""
        if self.model is None:
            raise ValueError("No model to save!")

        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        model_path = f'{self.mlp_dir}{model_name}_{timestamp}.keras'

        # Save model
        self.model.save(model_path)

        # Save metadata
        if metadata:
            metadata_path = f'{self.mlp_dir}{model_name}_{timestamp}_metadata.json'
            import json
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=2)

        print(f"✅ Model saved:")
        print(f"   Model: {model_path}")
        if metadata:
            print(f"   Metadata: {metadata_path}")

        return model_path

    def save_training_checkpoint(self, model, history, epoch, metadata=None):
        """Save training checkpoint for resumability"""
        checkpoint_data = {
            'epoch': epoch,
            'model_state': 'saved_separately',  # Model saved as .keras file
            'history': {key: list(values) for key, values in history.history.items()},
            'transformer_params': {
                'mean_': self.transformer.mean_.tolist() if hasattr(self.transformer, 'mean_') else None,
                'scale_': self.transformer.scale_.tolist() if hasattr(self.transformer, 'scale_') else None,
                'fitted': hasattr(self.transformer, 'mean_')
            },
            'is_fitted': self.is_fitted,
            'metadata': metadata or {}
        }

        checkpoint_file = f"{self.checkpoint_dir}training_checkpoint.json"
        model_file = f"{self.checkpoint_dir}model_checkpoint.keras"

        # Save checkpoint data
        import json
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f, indent=2)

        # Save model
        model.save(model_file)

        print(f"💾 Checkpoint saved at epoch {epoch}")
        return checkpoint_file, model_file

    def load_training_checkpoint(self):
        """Load training checkpoint for resuming"""
        checkpoint_file = f"{self.checkpoint_dir}training_checkpoint.json"
        model_file = f"{self.checkpoint_dir}model_checkpoint.keras"

        if not os.path.exists(checkpoint_file) or not os.path.exists(model_file):
            print("📝 No checkpoint found - starting fresh training")
            return None

        try:
            import json
            with open(checkpoint_file, 'r') as f:
                checkpoint_data = json.load(f)

            # Load model
            self.model = keras.models.load_model(model_file)

            # Restore transformer
            if checkpoint_data['transformer_params']['fitted']:
                self.transformer.mean_ = np.array(checkpoint_data['transformer_params']['mean_'])
                self.transformer.scale_ = np.array(checkpoint_data['transformer_params']['scale_'])
                self.transformer.n_features_in_ = len(self.transformer.mean_)

            # Restore state
            self.is_fitted = checkpoint_data['is_fitted']

            print(f"✅ Checkpoint loaded from epoch {checkpoint_data['epoch']}")
            return checkpoint_data

        except Exception as e:
            print(f"❌ Error loading checkpoint: {e}")
            return None

    def check_training_completion(self):
        """Check if training is already completed"""
        completion_file = f"{self.mlp_dir}training_completed.json"

        if os.path.exists(completion_file):
            try:
                import json
                with open(completion_file, 'r') as f:
                    completion_data = json.load(f)

                # Check if model file exists
                model_path = completion_data.get('model_path')
                if model_path and os.path.exists(model_path):
                    print("✅ Training already completed!")
                    print(f"   Model: {os.path.basename(model_path)}")
                    print(f"   Test R²: {completion_data.get('test_r2', 'N/A')}")
                    print(f"   Completed: {completion_data.get('completion_time', 'N/A')}")
                    return completion_data
            except Exception as e:
                print(f"⚠️ Error reading completion file: {e}")

        return None

    def mark_training_completed(self, results):
        """Mark training as completed with results"""
        completion_data = {
            'completion_time': pd.Timestamp.now().isoformat(),
            'model_path': results.get('model_path'),
            'test_r2': results['test_metrics']['r2'],
            'test_mae': results['test_metrics']['mae'],
            'train_samples': results['metadata']['training_samples'],
            'architecture': results['metadata']['architecture'],
            'epochs_trained': results['metadata']['epochs_trained']
        }

        completion_file = f"{self.mlp_dir}training_completed.json"
        import json
        with open(completion_file, 'w') as f:
            json.dump(completion_data, f, indent=2)

        print(f"📋 Training completion marked")

    def train_initial_mlp_resumable(self, x_train, y_train, x_val, y_val, epochs=1000,
                                   batch_size=64, architecture='default',
                                   checkpoint_every=50):
        """Enhanced initial training with resumability"""
        print("\n🚀 Training initial MLP with resumability...")

        # Check for existing checkpoint
        checkpoint_data = self.load_training_checkpoint()
        start_epoch = 0
        initial_history = None

        if checkpoint_data:
            start_epoch = checkpoint_data['epoch']
            initial_history = checkpoint_data['history']
            print(f"🔄 Resuming training from epoch {start_epoch}")
        else:
            # Build new model
            self.model = self.build_mlp(x_train.shape[1], architecture)

        # Callbacks
        checkpoint_path = f"{self.mlp_dir}best_model.weights.h5"

        # Custom callback for periodic checkpointing
        class ResumeCallback(keras.callbacks.Callback):
            def __init__(self, agent, checkpoint_every, metadata):
                self.agent = agent
                self.checkpoint_every = checkpoint_every
                self.metadata = metadata

            def on_epoch_end(self, epoch, logs=None):
                if (epoch + 1) % self.checkpoint_every == 0:
                    self.agent.save_training_checkpoint(
                        self.model,
                        self,
                        epoch + 1,
                        self.metadata
                    )

        callbacks = [
            ModelCheckpoint(
                checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            ),
            EarlyStopping(
                patience=50,
                verbose=1,
                min_delta=0.0001,
                monitor='val_loss',
                mode='min',
                restore_best_weights=True
            ),
            ResumeCallback(
                self,
                checkpoint_every,
                {'architecture': architecture, 'total_epochs': epochs}
            )
        ]

        # Calculate remaining epochs
        remaining_epochs = epochs - start_epoch

        if remaining_epochs > 0:
            print(f"🏃 Training for {remaining_epochs} more epochs...")

            # Training
            history = self.model.fit(
                x_train, y_train,
                validation_data=(x_val, y_val),
                epochs=remaining_epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=1,
                initial_epoch=start_epoch
            )

            # Combine with previous history if resuming
            if initial_history:
                for key in history.history:
                    if key in initial_history:
                        history.history[key] = initial_history[key] + history.history[key]
        else:
            print("✅ Training already completed!")
            # Create dummy history for consistency
            history = type('History', (), {'history': initial_history or {}})()

        self.is_fitted = True
        print("✅ Initial MLP training complete!")

        # Clean up checkpoint files
        self.cleanup_training_checkpoints()

        # Plot training history
        if hasattr(history, 'history') and history.history:
            self.plot_training_history(history)

        return history

    def cleanup_training_checkpoints(self):
        """Clean up temporary checkpoint files after successful training"""
        try:
            checkpoint_files = [
                f"{self.checkpoint_dir}training_checkpoint.json",
                f"{self.checkpoint_dir}model_checkpoint.keras"
            ]

            for file in checkpoint_files:
                if os.path.exists(file):
                    os.remove(file)

            print("🧹 Cleaned up training checkpoints")
        except Exception as e:
            print(f"⚠️ Could not clean up checkpoints: {e}")

    def run_complete_pipeline_resumable(self, remove_outliers=True, contamination=0.1,
                                      architecture='default', epochs=1000,
                                      auto_save=True, force_retrain=False):
        """
        Complete MLP training pipeline with resumability
        """
        print("="*80)
        print("🚀 STARTING RESUMABLE MLP TRAINING PIPELINE")
        print("="*80)

        # Check if training already completed (unless forcing retrain)
        if not force_retrain:
            completion_data = self.check_training_completion()
            if completion_data:
                # Load completed model
                model_path = completion_data['model_path']
                if os.path.exists(model_path):
                    self.model = keras.models.load_model(model_path)
                    self.is_fitted = True

                    print("🎉 Using already completed training!")
                    return {
                        'model': self.model,
                        'model_path': model_path,
                        'test_r2': completion_data['test_r2'],
                        'status': 'already_completed',
                        'completion_time': completion_data['completion_time']
                    }

        # Step 1: Load VAE outputs
        x, y = self.load_vae_outputs()
        if x is None:
            return None

        # Step 2: Analyze data
        self.analyze_data_distribution(y)

        # Step 3: Remove outliers (optional)
        if remove_outliers:
            x, y = self.remove_outliers(x, y, contamination)

        # Step 4: Prepare data
        x_train, x_val, x_test, y_train, y_val, y_test = self.prepare_data(x, y)

        # Step 5: Train model (resumable)
        history = self.train_initial_mlp_resumable(
            x_train, y_train, x_val, y_val,
            epochs=epochs, architecture=architecture
        )

        # Step 6: Comprehensive evaluation
        train_metrics = self.evaluate_model_comprehensive(x_train, y_train, "Training")
        val_metrics = self.evaluate_model_comprehensive(x_val, y_val, "Validation")
        test_metrics = self.evaluate_model_comprehensive(x_test, y_test, "Test")

        # Step 7: Save model with metadata
        model_path = None
        metadata = None

        if auto_save:
            metadata = {
                'architecture': architecture,
                'training_samples': len(x_train),
                'validation_samples': len(x_val),
                'test_samples': len(x_test),
                'train_r2': train_metrics['r2'],
                'val_r2': val_metrics['r2'],
                'test_r2': test_metrics['r2'],
                'epochs_trained': len(history.history.get('loss', [])),
                'outlier_removal': remove_outliers,
                'contamination': contamination if remove_outliers else None
            }
            model_path = self.save_model_with_metadata("resumable_pipeline", metadata)

        # Step 8: Mark training as completed
        results = {
            'model': self.model,
            'model_path': model_path,
            'history': history,
            'train_metrics': train_metrics,
            'val_metrics': val_metrics,
            'test_metrics': test_metrics,
            'metadata': metadata,
            'status': 'newly_completed'
        }

        if auto_save:
            self.mark_training_completed(results)

        print("="*80)
        print("✅ RESUMABLE MLP PIPELINE COMPLETE!")
        print(f"   Final Test R²: {test_metrics['r2']:.4f}")
        print(f"   Final Test MAE: {test_metrics['mae']:.4f}")
        print(f"   Accuracy within ±1: {test_metrics['accuracy_1']:.1f}%")
        print("="*80)

        return results

    def train_incremental_mlp(self, x_new, y_new, epochs=100, learning_rate_factor=0.1):
        """Enhanced incremental training"""
        if not self.is_fitted:
            raise ValueError("Must train initial model first!")

        print("\n🔄 Starting incremental training...")

        # Transform new target data
        y_new_transformed = self.transformer.transform(y_new.reshape(-1,1)).flatten()

        # Reduce learning rate for incremental training
        current_lr = self.model.optimizer.learning_rate
        new_lr = current_lr * learning_rate_factor
        self.model.optimizer.learning_rate = new_lr
        print(f"   Reduced learning rate to {new_lr:.6f}")

        # Split new data for validation
        x_train_new, x_val_new, y_train_new, y_val_new = train_test_split(
            x_new, y_new_transformed, test_size=0.2, random_state=42
        )

        # Incremental training with validation
        history = self.model.fit(
            x_train_new, y_train_new,
            validation_data=(x_val_new, y_val_new),
            epochs=epochs,
            batch_size=64,
            verbose=1
        )

        print("✅ Incremental training complete!")
        return history

# Integration with VAE Pipeline
def run_integrated_vae_mlp_pipeline(data_path, target_samples=350000):
    """
    Complete integrated pipeline: VAE → MLP
    """
    print("="*80)
    print("🎯 INTEGRATED VAE-MLP PIPELINE")
    print("="*80)

    # Step 1: Run VAE pipeline (if needed)
    from complete_vae_smart import VAEDataGenerator  # Import your VAE class

    vae_generator = VAEDataGenerator()
    vae_results = vae_generator.run_smart_pipeline(data_path, target_samples, show_visualization=False)

    if vae_results is None:
        print("❌ VAE pipeline failed!")
        return None

    # Step 2: Run MLP pipeline
    mlp_agent = EnhancedMLPAgent()
    mlp_results = mlp_agent.run_complete_pipeline()

    if mlp_results is None:
        print("❌ MLP pipeline failed!")
        return None

    print("="*80)
    print("🎉 INTEGRATED PIPELINE COMPLETE!")
    print(f"   VAE: {vae_results['synthetic_data'].shape[0]} synthetic samples")
    print(f"   MLP: R² = {mlp_results['test_metrics']['r2']:.4f}")
    print("="*80)

    return {
        'vae_results': vae_results,
        'mlp_results': mlp_results
    }

# Usage Examples
if __name__ == "__main__":

    # Option 1: Resumable MLP training (RECOMMENDED)
    mlp_agent = EnhancedMLPAgent()
    results = mlp_agent.run_complete_pipeline_resumable(
        remove_outliers=True,
        contamination=0.1,
        architecture='default',  # or 'deep', 'wide'
        epochs=1000,
        force_retrain=False  # Set to True to force restart
    )

    # Option 2: Force retrain from scratch
    # results = mlp_agent.run_complete_pipeline_resumable(force_retrain=True)

    # Option 3: Original pipeline (no resumability)
    # results = mlp_agent.run_complete_pipeline()

    # Option 4: Integrated VAE-MLP pipeline
    # integrated_results = run_integrated_vae_mlp_pipeline(
    #     r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-AUTO.npy',
    #     target_samples=350000
    # )

Epoch 1/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.8964 - mean_squared_error: 0.8964 - val_loss: 0.7946 - val_mean_squared_error: 0.7946
Epoch 2/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.8044 - mean_squared_error: 0.8044 - val_loss: 0.7901 - val_mean_squared_error: 0.7901
Epoch 3/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.7842 - mean_squared_error: 0.7842 - val_loss: 0.7714 - val_mean_squared_error: 0.7714
Epoch 4/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.7728 - mean_squared_error: 0.7728 - val_loss: 0.7566 - val_mean_squared_error: 0.7566
Epoch 5/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7643 - mean_squared_error: 0.7643 - val_loss: 0.7566 - val_mean_squared_error: 0.7566
Epoch 6/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7626 - mean_squared_error: 0.7626 - val_loss: 0.7513 - val_mean_squared_error: 0.7513
Epoch 7/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7563 - mean_squared_e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')